In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from os import listdir
from scipy import stats
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import tqdm
import statsmodels.api as sm
import statsmodels.formula.api as smf
%matplotlib inline

In [ ]:
# Global plotting constants.
sns.set(style="white", font_scale=1.5, font="Liberation Sans")
SURPRISAL_COLOR = "steelblue"
MARKERS = ["o", "v", "s", "d"] # use markers to differentiate between datasets

DATASET_NAMES = {
    "rx22": "Ronai & Xiang (2022)",
    "pvt21": "Pankratz & van Tiel (2021)",
    "g18": "Gotzner et al. (2018)",
    "vt16": "van Tiel et al. (2016)"
}

# Helper functions

In [ ]:
def render(file, fig_dir="figures"):
    path = f"{fig_dir}/{file}"
    plt.savefig(path, dpi=300, bbox_inches="tight")
    print(f"Rendered figure to {path}")

def regplt(df, x, y, label_name="Weaker", annot=True, **kwargs):
    pearsonr, pearsonp = stats.pearsonr(df[x], df[y])
    print(f"Pearson r={pearsonr}, p={pearsonp}")
    spearmanr, spearmanp = stats.spearmanr(df[x], df[y])
    print(f"Spearman r={spearmanr}, p={spearmanp}")

    # Plot
    ax = sns.regplot(data=df, x=x, y=y, **kwargs)
    plt.gcf().set_size_inches(8,6)
    if annot:
        for i, row in df.iterrows():
            ax.text(row[x], row[y], row[label_name], size="xx-small")
    return ax, pearsonr, pearsonp

In [ ]:
# Read a bunch of model outputs in TSV SyntaxGym format.
def read_results(result_dir):
    files = sorted([f for f in listdir(result_dir) if f.endswith("gpt2.tsv")])
    dfs = []
    sentence_dfs = []
    for f in files:
        try:
            _df = pd.read_csv(result_dir + "/" + f, sep="\t")
            weak, strong, model = f[:-4].split("_")
            scale_id = weak + "/" + strong
            _df["scale_id"] = scale_id
            _df["model"] = model
            dfs.append(_df)
            
            g = _df.fillna("").groupby("item_number").agg({'content': " ".join}).reset_index()
            g["scale_id"] = scale_id
            g["content"] = g.content.str.replace(" .", ".", regex=False).replace("\s+,", ",", regex=True)
            sentence_dfs.append(g)
        except:
            print(f"Skipping {f}")
    df = pd.concat(dfs)
    sentence_df = pd.concat(sentence_dfs)
    return df, sentence_df

def get_strong_surprisal_df(surprisal_df, region_numbers=[5]):
    print("Only considering surprisal at the following regions:", region_numbers)
    
    # Get subset of surprisals that correspond to specified region numbers.
    strong_surprisal_df = surprisal_df[surprisal_df.region_number.isin(region_numbers)].fillna("")
    
    # Sum surprisal and concatenate string content over regions of interest.
    strong_surprisal_df = strong_surprisal_df.groupby(
        ["scale_id", "item_number"]
    ).agg(
        {"content": " ".join, "metric_value": np.sum}
    ).reset_index()
    
    # Deal with edge case (removing extra space before EOS period).
    strong_surprisal_df["content"] = strong_surprisal_df.content.str.replace(" .", ".", regex=False)
    
    # Add unique ID for each item.
    strong_surprisal_df["unique_id"] = strong_surprisal_df.scale_id + "_" + strong_surprisal_df.item_number.astype(str)
    
    return strong_surprisal_df

In [ ]:
def process_model_outputs(result_dir, stimuli_file, model="gpt2", template=None):
    # Read model outputs and only consider surprisal at target region.
    df, sentence_df = read_results(result_dir)
    strong_surp = get_strong_surprisal_df(df).reset_index()
    
    # Read original data file with stimuli and human empirical results.
    print(f"Reading stimuli from {stimuli_file}")
    stims = pd.read_csv(stimuli_file)
    if "scale_id" not in list(stims):
        try:
            stims["scale_id"] = stims.Weaker + "/" + stims.Stronger
        except:
            stims["scale_id"] = stims.weak_adj + "/" + stims.strong_adj

    # Add surprisal results.
    print("Adding strong scalemate surprisal results")
    strong_surp["strong"] = strong_surp.scale_id.str.split('/').str[1]
    strong_surp["is_strong_scalemate"] = strong_surp.apply(
        lambda r: r.content==r.strong+"." or r.content.split()[0]==r.strong or f" {r.strong}" in r.content, 
        axis=1
    )
    s = strong_surp[strong_surp.is_strong_scalemate].set_index("scale_id")
    if template is None:
        surprisal_col_name = f"surprisal_{model}"
    else:
        surprisal_col_name = f"surprisal_{model}_template{template}"
    stims[surprisal_col_name] = stims.apply(
        lambda row: s.loc[row.scale_id].metric_value if row.scale_id in s.index else None, 
        axis=1
    )
    
    # Standardize dependent variable name across datasets.
    stims = stims.rename(columns={
        "SI percent (Exp 1)": "si_rate",
        "SI_rate": "si_rate",
        "si_nonneutral": "si_rate"
    })
    
    return stims, strong_surp, sentence_df

# Read data

In [ ]:
# Ronai & Xiang (2022)
rx, rx_strong, rx_sents = process_model_outputs(
    "./model_results/rx22", 
    "./human_data/rx22.csv"
)
rx.head()

In [ ]:
# Pankratz & van Tiel (2021)
pvt, pvt_strong, pvt_sents = process_model_outputs(
    "./model_results/pvt21", 
    "./human_data/pvt21.csv"
)
pvt["si_rate"] *= 100
pvt["pos"] = "adj" # all adjectival
pvt.head()

In [ ]:
# Gotzner et al. (2018)
gz, gz_strong, gz_sents = process_model_outputs(
    "./model_results/g18", 
    "./human_data/g18.csv"
)
gz["si_rate"] *= 100
gz.head()

In [ ]:
# van Tiel et al. (2016)
vt1, vt_strong1, vt_sents1 = process_model_outputs(
    "./model_results/vt16/template1", "./human_data/vt16.csv", template=1
)
vt2, vt_strong2, vt_sents2 = process_model_outputs(
    "./model_results/vt16/template2", "./human_data/vt16.csv", template=2
)
vt3, vt_strong3, vt_sents3 = process_model_outputs(
    "./model_results/vt16/template3", "./human_data/vt16.csv", template=3
)
vt = vt1.copy()
metric = "surprisal"
vt[f"{metric}_gpt2_template2"] = vt2[f"{metric}_gpt2_template2"]
vt[f"{metric}_gpt2_template3"] = vt3[f"{metric}_gpt2_template3"]
vt[f"{metric}_gpt2"] = vt.apply(
    lambda r: np.mean([r[f"{metric}_gpt2_template{template}"] for template in [1,2,3]]),
    axis=1
)
vt.head()

# Bookkeeping

In [ ]:
n_unique_scales = len(set(
    rx.dropna().scale_id.unique().tolist() + 
    pvt.dropna().scale_id.unique().tolist() + 
    gz.dropna().scale_id.unique().tolist() + 
    vt.dropna().scale_id.unique().tolist()
))
print(f"{n_unique_scales} total unique scales")

In [ ]:
DATASETS = [
    ("Ronai & Xiang (2022)", rx),
    ("Pankratz & van Tiel (2021)", pvt),
    ("Gotzner et al. (2018)", gz),
    ("van Tiel et al. (2016)", vt)
]

for name, df in DATASETS:
    print(name)
    print(df.dropna().pos.value_counts())
    print("Total unique scales:", df.dropna().scale_id.nunique())
    print("~"*20)

In [ ]:
from collections import defaultdict
UNIQUE_SCALES = defaultdict(list)
for pos in ["adj", "adv", "verb"]:
    for _, df in DATASETS:
        scales = df[df.pos==pos].dropna().scale_id.tolist()
        UNIQUE_SCALES[pos] += scales
UNIQUE_SCALES = {pos: set(scales) for pos, scales in UNIQUE_SCALES.items()}
for pos, scales in UNIQUE_SCALES.items():
    print(pos, len(scales))

# Main results

## Figure 3a: String-based surprisal

In [ ]:
# Set variables for plotting.
x = "surprisal_gpt2"
metric = x.split("_")[0]
y = "si_rate"

# Set up subplots.
fig, axes = plt.subplots(nrows=1, ncols=len(DATASETS), sharey=True, sharex=False)

# Iterate over datasets and axes.
for dataset_idx, (dataset_name, df) in enumerate(DATASETS):
    print(dataset_name)
    ax, r, p = regplt(
        df.dropna(), 
        x, 
        y, 
        ax=axes[dataset_idx], 
        color=SURPRISAL_COLOR, 
        marker=MARKERS[dataset_idx],
        annot=False
    )
    ax.set_title(dataset_name, pad=26, size=16)
    ax.text(
        0.5, 1.07, f"$r = {r:.3f}, p = {p:.3f}$", color="dimgrey",
        size=14, transform=ax.transAxes, horizontalalignment='center', verticalalignment='center'
    )
    if dataset_idx == 0:
        ax.set_ylabel("Human SI rate")
    else:
        ax.set_ylabel("")
    ax.set_xlabel("")
    print("="*80)

fig.text(0.5, -0.1, "Surprisal of strong scalemate", ha='center', size=18)
plt.subplots_adjust(wspace=0.2)
fig.set_size_inches(13,3)
render(f"cross-scale_{metric}.pdf")
plt.show()

Footnote 9: Recompute results after removing outlier from Gotzner et al.'s (2018) data.

In [ ]:
y = "si_rate"
x = "surprisal_gpt2"
for dataset_idx, (dataset_name, df) in enumerate(DATASETS):
    if dataset_name == "Gotzner et al. (2018)":
        # Remove outliers
        outliers = df.dropna()[(np.abs(stats.zscore(df.dropna()[x])) >= 3)]
        print(outliers)
        tmp = df.dropna()[(np.abs(stats.zscore(df.dropna()[x])) < 3)].copy()
        r, p = stats.pearsonr(tmp[x], tmp[y])
        print(f"Without outliers: r={r}, p={p}")

## Figure 3b: Weighted average surprisal

Prep data: get distributions over full alternative set.

In [ ]:
def get_dists_ranks(strong, exclusions=[], top_k=None):
    g = strong.groupby("scale_id").apply(
        lambda x: x.sort_values(by="metric_value", ascending=True)
    ).drop("scale_id", axis=1).reset_index()
    dists, ranks = [], []
    for scale_id in sorted(g.scale_id.unique()):
        strong_scalemate = scale_id.split("/")[1]
        if all(e != scale_id.split("/")[0] for e in exclusions):
            rows = g[g.scale_id==scale_id].reset_index()
            # Get rank of strong scalemate.
            surprisals = rows.metric_value
            scalemates = rows.content.tolist()
            try:
                rank = next(
                    (i for i in range(len(scalemates)) if scalemates[i] == strong_scalemate+"." or scalemates[i].split()[0] == strong_scalemate)
                ) + 1
                ranks.append(dict(
                    scale_id=scale_id,
                    rank=rank,
                    surprisal=surprisals[rank-1]
                ))
            except:
                print("skipping", scale_id)
                continue
            
            # Get scores for top k scalemates.
            if top_k is not None:
                iterrows = list(rows.iterrows())[:top_k]
            else:
                iterrows = list(rows.iterrows())
            for i, row in iterrows:
                surprisal = row.metric_value
                dists.append(dict(
                    scale_id=scale_id,
                    rank=i+1,
                    scalemate=row.content,
                    surprisal=row.metric_value,
                    prob=np.exp(-row.metric_value),
                    is_strong_scalemate=(row.content==strong_scalemate+"." or row.content.split()[0]==strong_scalemate)
                ))
    
    dists = pd.DataFrame(dists)
    ranks = pd.DataFrame(ranks)
    return dists, ranks

def add_rank_data(ranks, orig, template=None):
    if template is None:
        col_name = "strong_scalemate_rank_gpt2"
    else:
        col_name = f"strong_scalemate_rank_gpt2_template{template}"
                
    for i, row in orig.iterrows():
        if row.scale_id in ranks.scale_id.unique():
            rank = ranks[ranks.scale_id==row.scale_id].squeeze()["rank"]
            orig.loc[i, col_name] = rank
    orig[f"log_{col_name}"] = np.log(orig[col_name])
    return orig

In [ ]:
# Prep data for Ronai & Xiang (2022).
rx_dists, rx_ranks = get_dists_ranks(rx_strong)
print(rx_ranks.head())
rx = add_rank_data(rx_ranks, rx)

# Prep data for Pankratz & van Tiel (2021).
pvt_dists, pvt_ranks = get_dists_ranks(pvt_strong)
print(pvt_ranks.head())
pvt = add_rank_data(pvt_ranks, pvt)

# Prep data for Gotzner et al. (2018)
gz_dists, gz_ranks = get_dists_ranks(gz_strong)
print(gz_ranks.head())
gz = add_rank_data(gz_ranks, gz)

# Prep data for van Tiel et al. (2016)
vt_dists1, vt_ranks1 = get_dists_ranks(vt_strong1)
vt_dists2, vt_ranks2 = get_dists_ranks(vt_strong2)
vt_dists3, vt_ranks3 = get_dists_ranks(vt_strong3)
vt = add_rank_data(vt_ranks1, vt, template=1)
vt = add_rank_data(vt_ranks2, vt, template=2)
vt = add_rank_data(vt_ranks3, vt, template=3)
vt.head()

Prep word vectors to get similarity-based weights.

In [ ]:
# Read word vectors. This may take a while (~1min).
glove_file = "glove.6B.300d.txt"
with open(glove_file, 'r') as f:
    glove_vectors = {}
    for line in f:
        vals = line.rstrip().split(' ')
        glove_vectors[vals[0]] = np.array([float(x) for x in vals[1:]]).reshape(1, -1)
print("Successfully loaded GloVe vectors")

In [ ]:
# Helper function for getting cosine similarity between word vectors
def get_sim(w1, w2, vectors):
    try:
        v1, v2 = vectors[w1], vectors[w2]
        sim = cosine_similarity(v1, v2)
        return sim[0][0]
    except:
        return None

# Helper function for getting similarity for top k scalemates and strong scalemate
def get_sim_scalemates(dists, vectors, topk=None):
    if topk is not None:
        dists = dists[dists["rank"]<=topk]
    dists["cosine_sim_strong"] = dists.apply(
        lambda row: get_sim(row.scalemate, row.scale_id.split("/")[1], vectors),
        axis=1
    )
    return dists

# Get weighted average surprisal over full alternative set
def get_weighted_avg_surprisal(dists, vectors, **kwargs):
    print("Getting similarity scores")
    dists = get_sim_scalemates(dists, vectors, **kwargs)
    print("Computing weighted average surprisals")
    data = []
    sims_data = []
    for scale_id in dists.scale_id.unique():
        d = dists[dists.scale_id==scale_id]
        probs = d[~d.cosine_sim_strong.isna()].prob
        sims = d[~d.cosine_sim_strong.isna()].cosine_sim_strong
        sims_data.append(dict(scale_id=scale_id, sims=sims.tolist()))
        weights = sims + 1 # translate from [-1, 1] to [0, 2] - avoid weirdness of negative weights
        if sum(sims) != 0:
            wavg_surp = -np.log(np.average(probs, weights=weights))
            data.append(dict(
                scale_id=scale_id,
                weighted_avg_surprisal=wavg_surp
            ))
    return pd.DataFrame(data), pd.DataFrame(sims_data)

Computed weighted-average surprisal.

In [ ]:
def get_wavg(*args, **kwargs):
    wavg, sims = get_weighted_avg_surprisal(*args, **kwargs)
    wavg = wavg.set_index("scale_id").weighted_avg_surprisal
    return wavg, sims

topk = None
embs = glove_vectors 
all_sims = []

print("Ronai & Xiang (2022)")
wavg, sims = get_wavg(rx_dists, embs, topk=topk)
sims["dataset"] = "rx22"
all_sims.append(sims)
for i, row in tqdm(rx.iterrows(), total=len(rx.index)):
    rx.loc[i, "weighted_avg_surprisal"] = wavg.loc[row.scale_id] if row.scale_id in wavg else None
    
print("Pankratz & van Tiel (2021)")
wavg, sims = get_wavg(pvt_dists, embs, topk=topk)
sims["dataset"] = "pvt21"
all_sims.append(sims)
for i, row in tqdm(pvt.iterrows(), total=len(pvt.index)):
    pvt.loc[i, "weighted_avg_surprisal"] = wavg.loc[row.scale_id] if row.scale_id in wavg else None

print("Gotzner et al. (2018)")
wavg, sims = get_wavg(gz_dists, embs, topk=topk)
sims["dataset"] = "gz18"
all_sims.append(sims)
for i, row in tqdm(gz.iterrows(), total=len(gz.index)):
    gz.loc[i, "weighted_avg_surprisal"] = wavg.loc[row.scale_id] if row.scale_id in wavg else None
    
print("van Tiel et al. (2016)")
wavg1, sims1 = get_wavg(vt_dists1, embs, topk=topk)
wavg2, sims2 = get_wavg(vt_dists2, embs, topk=topk)
wavg3, sims3 = get_wavg(vt_dists3, embs, topk=topk)
sims["dataset"] = "vt16"
all_sims.append(sims)
for i, row in tqdm(vt.iterrows(), total=len(vt.index)):
    vals = [wavg.loc[row.scale_id] for wavg in [wavg1, wavg2, wavg3] if row.scale_id in wavg]
    vt.loc[i, "weighted_avg_surprisal"] = np.mean(vals)

In [ ]:
# Set variables for plotting.
x = "weighted_avg_surprisal"
y = "si_rate"

# Set up subplots.
fig, axes = plt.subplots(nrows=1, ncols=len(DATASETS), sharey=True, sharex=False)

# Iterate over datasets and axes.
for dataset_idx, (dataset_name, df) in enumerate(DATASETS):
    ax, r, p = regplt(
        df.dropna(), 
        x, 
        y, 
        ax=axes[dataset_idx], 
        color=SURPRISAL_COLOR, 
        marker=MARKERS[dataset_idx],
        annot=False
    )
    ax.set_title(dataset_name, pad=26, size=16)
    p_str = f"{p:.4f}" if p < 0.001 else f"{p:.3f}"
    ax.text(
        0.5, 1.07, f"$r = {r:.3f}, p = {p_str}$", color="dimgrey",
        size=14, transform=ax.transAxes, horizontalalignment='center', verticalalignment='center'
    )
    if dataset_idx == 0:
        ax.set_ylabel("Human SI rate")
    else:
        ax.set_ylabel("")
    ax.set_xlabel("")
    print("="*80)

fig.text(0.5, -0.1, "Weighted average surprisal", ha='center', size=18)
plt.subplots_adjust(wspace=0.2)
fig.set_size_inches(13,3)
render(f"cross-scale_{x}.pdf")
plt.show()

## Table 4: Multivariate analysis & ANOVA

In [ ]:
def get_sig_code(p):
    if p < 0.001:
        return "***"
    elif p < 0.01:
        return "**"
    elif p < 0.05:
        return "*"
    elif p < 0.1:
        return "."
    else:
        return ""

def fit_reg_models(df):
    # Initialize list of model results.
    all_results = []
    
    # Two models: one with both surprisal predictors, and one with neither.
    formula_main = "si_rate ~ 1 + csurprisal + cweighted_avg_surprisal"
    formula_no_effects = "si_rate ~ 1"
    m_main = smf.ols(formula_main, data=df).fit()
    m_no_effects = smf.ols(formula_no_effects, data=df).fit()
    
    # Record results.
    for i, m in enumerate([m_main, m_no_effects]):
        coeffs = m.params.to_frame().reset_index().set_axis(['variable', 'coeff'], axis=1).set_index('variable')
        pvals = m.pvalues.to_frame().reset_index().set_axis(['variable', 'pval'], axis=1).set_index('variable')
        results = coeffs.join(pvals).reset_index()
        # results = results[results.variable != "Intercept"]
        results["sig"] = results["pval"] < 0.05
        results["sig_code"] = results["pval"].apply(get_sig_code)
        results["formula"] = [formula_main, formula_no_effects][i]
        all_results.append(results)
    
    # ANOVA
    anova = sm.stats.anova_lm(m_no_effects, m_main, typ=1)
    return anova, pd.concat(all_results)

stat_dfs = []
anova_dfs = []
for (dataset_name, df) in DATASETS:
    # Center the relevant variables.
    s = df.copy()
    s["csurprisal"] = (s.surprisal_gpt2 - s.surprisal_gpt2.mean())
    s["cweighted_avg_surprisal"] = (s.weighted_avg_surprisal - s.weighted_avg_surprisal.mean())

    # Fit models and perform ANOVA.
    anova, res = fit_reg_models(s)
    anova["dataset"] = dataset_name
    anova["sig"] = anova["Pr(>F)"] < 0.05
    anova["sig_code"] = anova["Pr(>F)"].apply(get_sig_code)
    res["dataset"] = dataset_name
    anova_dfs.append(anova)
    stat_dfs.append(res)
    
stat_df = pd.concat(stat_dfs)
anova_df = pd.concat(anova_dfs)

In [ ]:
stat_df

In [ ]:
anova_df

## Figure 4: Model surprisal vs. human completions

In [ ]:
ax, r, p = regplt(
    rx.dropna(), 
    "surprisal_gpt2", 
    "Acessibility (Exp 2)", 
    color=SURPRISAL_COLOR,
    annot=False,
    marker=MARKERS[0],
    scatter_kws=dict(s=40)
)
ax.set_xlabel("Surprisal of strong scalemate")
ax.set_ylabel("\"Accessibility\" score")
ax.set_title(f"Ronai & Xiang (2022)", pad=26)
ax.text(
    0.5, 1.07, f"$r = {r:.3f}, p = {p:.3f}$", color="dimgrey",
    size=15, transform=ax.transAxes, horizontalalignment='center', verticalalignment='center'
)
plt.gcf().set_size_inches(2.837,3)
render("accessibility_rx22.pdf")
plt.show()

# Qualitative analysis

## Figure 5: top-ranked alternatives

In [ ]:
include = ["big/enormous", "largely/totally", "hard/unsolvable"] 
g = sns.catplot(
    data=rx_dists[(rx_dists.scale_id.isin(include))&(rx_dists["rank"]<=5)], 
    x="prob", y="scalemate", 
    col="scale_id", kind="bar",
    col_order=include,
    sharex=False, sharey=False,
    height=3, aspect=1.5
)
g.set_axis_labels("P(alternative | context)", "Alternative")
axes = g.axes[0]
for ax in axes:
    ax.set_title("")
render("top_ranked_alts_rx22_notitles.pdf")